# Implementación Modelo DQN

In [ ]:
!pip install torch --upgrade

In [ ]:
!pip install tensorflow

In [ ]:
from tqdm import tqdm
import os
import numpy as np
import pandas as pd
from typing import List, Any
from datetime import datetime
import random
from collections import deque
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Helper functions
def get_unix(date: str, time: str) -> int:
    format_date = "%d/%m/%Y"
    format_time = "%H:%M:%S.%f000"
    date_obj = datetime.strptime(date, format_date)
    time_obj = datetime.strptime(time, format_time)
    almost = datetime.combine(date_obj.date(), time_obj.time())
    return int(almost.timestamp())

class Tick:
    def __init__(self, line: str) -> None:
        splitted_line = line.split(',')
        self.date = splitted_line[0]
        self.time = splitted_line[1]
        self.bid = float(splitted_line[2].strip())
        self.ask = float(splitted_line[4].strip())
        self.unix_code = get_unix(splitted_line[0], splitted_line[1])

class Strategy:
    def __init__(self) -> None:
        pass
    def Von_Neuman(self, id: int, prices: List[float], tres_hold: float, acumulated_side_bool: bool, cant_part=8) -> int:
        prices = prices[::-1]
        if (id == 0) or (acumulated_side_bool is False):
            up_down = []
            if cant_part < len(prices):
                for index_groups in range(cant_part - 1):
                    try:
                        if prices[index_groups * 100] < prices[(index_groups + 1) * 100]:
                            up_down.append(1)
                        elif prices[index_groups * 100] > prices[(index_groups + 1) * 100]:
                            up_down.append(-1)
                        else:
                            up_down.append(0)
                    except IndexError:
                        up_down.append(0)
                try:
                    avg = sum(up_down) / len(up_down)
                    if avg > tres_hold:
                        return -1
                    else:
                        return 0
                except ZeroDivisionError:
                    return 0
            else:
                return 0
        elif (id > 0) and (acumulated_side_bool is True):
            up_down = []
            if cant_part < len(prices):
                for index_groups in range(cant_part - 1):
                    try:
                        if prices[index_groups * 10] < prices[(index_groups + 1) * 10]:
                            up_down.append(1)
                        elif prices[index_groups * 10] > prices[(index_groups + 1) * 10]:
                            up_down.append(-1)
                        else:
                            up_down.append(0)
                    except IndexError:
                        up_down.append(0)
                try:
                    avg = sum(up_down) / len(up_down)
                    if -tres_hold <= avg <= tres_hold:
                        return 0
                    elif avg > 0:
                        return -1
                    else:
                        return 1
                except ZeroDivisionError:
                    return 0
            else:
                return 0

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=500)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()
        self.update_counter = 0

    def _build_model(self):
        model = Sequential()
        model.add(Dense(8, input_dim=self.state_size, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state, verbose=0)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state, verbose=0)[0])
            target_f = self.model.predict(state, verbose=0)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=5, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

def compute_balance(orders_list: List[Any], tick: Tick) -> float:
    open_balance = 0
    for line in orders_list:
        if len(line) > 2:
            side = line[0]
            number_units = line[1]
            bid_price, ask_price = line[2], line[3]
            last_bid, last_ask = tick.bid, tick.ask
            profit_loss = 0
            if side == -1:
                profit_loss = number_units * last_bid - number_units * ask_price
            elif side == 1:
                profit_loss = number_units * bid_price - number_units * last_ask
            open_balance += profit_loss
    return open_balance * 1

def compute_drawdown(path_to_use: str, orders_list: List[Any], tick: Tick) -> float:
    if len(orders_list) > 1:
        last_balance = compute_balance(orders_list, tick)
        balances = []
        with open(path_to_use, 'r') as file_reader:
            for line in file_reader:
                balances.append(float(line.split(':')[6]))
        balances.append(last_balance)
        max_drawdown = 0
        drawdown = 0
        peak = balances[0]
        for balance in balances:
            if balance > peak:
                peak = balance
            drawdown = peak - balance
            if drawdown > max_drawdown:
                max_drawdown = drawdown
        return max_drawdown
    else:
        return 0

def Register(path_to_use: str, line: List[Any]) -> None:
    with open(path_to_use, 'a') as file_in_process:
        for value in line:
            file_in_process.writelines(f'{value}:')
        file_in_process.writelines('\n')

def file_content(path_to_use: str) -> str:
    if os.path.exists(path_to_use):
        with open(path_to_use, 'r') as file_in_process:
            return file_in_process.read()
    else:
        return ""

def time_signal_detector(current_tick: Tick, previous_tick: Tick, pivot_unix: int) -> bool:
    current_unix = current_tick.unix_code
    previous_unix = previous_tick.unix_code
    if current_unix <= pivot_unix:
        return True
    elif previous_unix <= pivot_unix < current_unix:
        return False

def State_detector(acumulated_side) -> bool:
    return acumulated_side < 0

dict_for_tres_hold = {0.07: 'A', 0.08: 'B', 0.09: 'C', 0.1: 'D', 0.15: 'E', 0.16: 'F', 0.17: 'G', 0.18: 'H', 0.19: 'I', 0.2: 'J', 0.25: 'K'}
dict_for_decision = {1: 'sell', 0: 'hold', -1: 'buy'}
time_frame = 616
dict_for_time_frames = {'616': 'P_0'}

def Process_day(path_of_current_day: str, day: str, Strat: Strategy, agent: DQNAgent, sum_side_by_day=0) -> str:
    list_for_balance = []
    prices = []
    index_for_balance = 0
    state_size = 5
    action_size = len(dict_for_tres_hold)
    batch_size = 8

    with open(path_of_current_day, 'r') as current_day:
        current_day_list = current_day.readlines()
        last_hour = int(((current_day_list[-2].split(','))[1].split(':'))[0]) + 1
        current_hour = 0

        current_minute = 0
        current_seconds = 0

        pivot_time = f'{current_hour:02}:{current_minute:02}:{current_seconds:02}.000000000'
        id = 0

        for index in tqdm(range(1, len(current_day_list)), desc=f'Processing {day}'):
            current_tick = Tick(current_day_list[index])
            previous_tick = Tick(current_day_list[index - 1])
            pivot_unix = get_unix(current_tick.date, pivot_time)
            if time_signal_detector(current_tick, previous_tick, pivot_unix):
                prices.append(np.random.choice([previous_tick.bid, current_tick.ask]))
            else:
                if len(prices) >= state_size:
                    state = np.reshape(prices[-state_size:], [1, state_size])
                    action = agent.act(state)
                    selected_tres_hold = list(dict_for_tres_hold.keys())[action]
                    side = Strat.Von_Neuman(id, prices, selected_tres_hold, State_detector(sum_side_by_day))
                    sum_side_by_day += side
                    reward = 0
                    if side != 0:
                        list_for_balance.append([side, 1, previous_tick.bid, previous_tick.ask])
                        P_L = compute_balance(list_for_balance, previous_tick)
                        drawdown = compute_drawdown(dict_for_time_frames[str(time_frame)] + '_' + day, list_for_balance, previous_tick)
                        reward = P_L  # Using profit/loss as reward
                        line_to_register = [id, dict_for_tres_hold[selected_tres_hold], dict_for_decision[side], 1, previous_tick.bid, previous_tick.ask, P_L, drawdown]
                        Register(dict_for_time_frames[str(time_frame)] + '_' + day, inverted_line.split(':'))
                        id += 1

                    next_state = np.reshape(prices[-state_size:], [1, state_size])
                    done = index == len(current_day_list) - 1
                    agent.remember(state, action, reward, next_state, done)
                    N = 1000
                    agent.update_counter += 1
                    if len(agent.memory) > batch_size and agent.update_counter % N == 0:
                        agent.replay(batch_size)

                current_seconds += 5
                if current_seconds >= 60:
                    current_seconds -= 60
                    current_minute += 1

                if current_minute >= 60:
                    current_minute -= 60
                    current_hour += 1

                if current_hour >= last_hour:
                    return dict_for_time_frames[str(time_frame)] + '_' + day

                pivot_time = f'{current_hour:02}:{current_minute:02}:{int(current_seconds):02}.000000000'
                prices = []
    return dict_for_time_frames[str(time_frame)] + '_' + day


folder = '/content/drive/My Drive/archivos'
folder_for_results = '/content/drive/My Drive/New_results'
archives_in_folder = os.listdir(folder)
for archive_name in archives_in_folder:
    if archive_name.endswith('.txt'):
        complete_route = os.path.join(folder, archive_name)
        print(f'Processing file: {complete_route}')
        Strat = Strategy()
        agent = DQNAgent(state_size=5, action_size=len(dict_for_tres_hold))
        path_of_analized_day = Process_day(complete_route, archive_name, Strat, agent)
        place_path = os.path.join(folder_for_results, path_of_analized_day)
        content = file_content(path_of_analized_day)
        if content.strip():
            with open(place_path, 'w') as archive:
                archive.write(content)
        if os.path.exists(path_of_analized_day):
            os.remove(path_of_analized_day)
        else:
            pass


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processing file: /content/drive/My Drive/archivos/04-12-2022.txt


Processing 04-12-2022.txt: 100%|█████████▉| 86478/86505 [00:17<00:00, 5023.76it/s]


Processing file: /content/drive/My Drive/archivos/22-12-2022.txt


Processing 22-12-2022.txt: 100%|█████████▉| 571494/571502 [02:24<00:00, 3957.41it/s]


Processing file: /content/drive/My Drive/archivos/26-12-2022.txt


...

Processing 03-03-2023.txt: 100%|█████████▉| 565186/565189 [02:03<00:00, 4570.92it/s]


Processing file: /content/drive/My Drive/archivos/27-01-2023.txt


Processing 27-01-2023.txt: 100%|█████████▉| 711294/711297 [02:25<00:00, 4893.62it/s]


Processing file: /content/drive/My Drive/archivos/11-01-2023.txt


Processing 11-01-2023.txt: 100%|█████████▉| 712041/712045 [03:04<00:00, 3857.79it/s]


In [ ]:
from google.colab import drive
import os
import pandas as pd

# Montar Google Drive
drive.mount('/content/drive')

# Definir los directorios
input_directory = "/content/drive/My Drive/New_results"
output_directory = "/content/drive/My Drive/summary"

# Crear el directorio de salida si no existe
os.makedirs(output_directory, exist_ok=True)

# Listar todos los archivos de entrada
input_files = [file for file in os.listdir(input_directory) if file.endswith('.txt')]

# Diccionario para almacenar los resúmenes
summary_dict = {}

# Procesar cada archivo
for file in input_files:
    file_path = os.path.join(input_directory, file)

    # Leer el archivo en un DataFrame
    df = pd.read_csv(file_path, header=None, delimiter=":")
    # Extraer el threshold (segunda columna) y balance (sexta columna)
    df['threshold'] = df[1]
    df['balance'] = df[6]

    # Sumar los balances para cada threshold
    threshold_sums = df.groupby('threshold')['balance'].sum().reset_index()

    # Agregar al diccionario de resúmenes
    for index, row in threshold_sums.iterrows():
        threshold = row['threshold']
        balance = row['balance']

        if threshold not in summary_dict:
            summary_dict[threshold] = 0
        summary_dict[threshold] += balance

# Escribir los resúmenes en archivos de salida
for threshold, total_balance in summary_dict.items():
    summary_file_path = os.path.join(output_directory, f"summary_{threshold}.txt")
    with open(summary_file_path, 'w') as summary_file:
        summary_file.write(f"Threshold: {threshold}\nTotal Balance: {total_balance}\n")

print("Los archivos de resumen se han creado en la carpeta 'summary'.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Los archivos de resumen se han creado en la carpeta 'summary'.


In [ ]:
from google.colab import drive
import os
import pandas as pd

# Montar Google Drive
drive.mount('/content/drive')

# Definir los directorios
input_directory = "/content/drive/My Drive/New_results"
output_directory = "/content/drive/My Drive/summary"

# Crear el directorio de salida si no existe
os.makedirs(output_directory, exist_ok=True)

# Listar todos los archivos de entrada
input_files = [file for file in os.listdir(input_directory) if file.endswith('.txt')]

# Diccionario para almacenar los resúmenes
summary_dict = {}

# Procesar cada archivo
for file in input_files:
    file_path = os.path.join(input_directory, file)

    # Leer el archivo en un DataFrame
    df = pd.read_csv(file_path, header=None, delimiter=":")

    # Extraer el threshold (segunda columna) y drawdown (última columna)
    df['threshold'] = df[1]
    df['drawdown'] = df[7]
    df['drawdown'] = df['drawdown'].abs()

    # Sumar los drawdowns para cada threshold
    threshold_sums = df.groupby('threshold')['drawdown'].sum().reset_index()

    # Agregar al diccionario de resúmenes
    for index, row in threshold_sums.iterrows():
        threshold = row['threshold']
        drawdown = row['drawdown']

        if threshold not in summary_dict:
            summary_dict[threshold] = 0
        summary_dict[threshold] += drawdown

# Escribir los resúmenes en archivos de salida
for threshold, total_drawdown in summary_dict.items():
    summary_file_path = os.path.join(output_directory, f"summary_drawdown_{threshold}.txt")
    with open(summary_file_path, 'w') as summary_file:
        summary_file.write(f"Threshold: {threshold}\nTotal Drawdown: {total_drawdown}\n")

print("Los archivos de resumen de drawdown se han creado en la carpeta 'summary'.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Los archivos de resumen de drawdown se han creado en la carpeta 'summary'.
